In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random

In [56]:
USER_AGENTS = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15'
    ]
headers = {
            'User-Agent': random.choice(USER_AGENTS),
            'Accept-Language': 'en-US, en;q=0.5'
        }

In [ ]:
url = "https://www.walmart.com/browse/beauty/cologne-for-men/1085666_133225_6873232?povid=Beauty_FragranceCP_Hubspokes_ShopByCat_Forhim_122624"

In [53]:
webpage = requests.get(url, headers=headers)
webpage.status_code

200

In [54]:
soup = BeautifulSoup(webpage.content, 'html.parser')

In [55]:
links = soup.find_all("a", class_='hide-sibling-opacity')


In [57]:
len(links)

12

In [37]:
link = links[0].get('href')

In [38]:
product_link = "https://www.walmart.com" + link
product_link

'https://www.walmart.com/ip/Equate-24-Hour-Allergy-Cetirizine-Hydrochloride-Tablets-10-mg-45-Count/10899001?classType=VARIANT&athbdg=L1300'

In [39]:
new_webpage = requests.get(product_link, headers=headers)
new_webpage.status_code

200

In [40]:
new_soup = BeautifulSoup(new_webpage.content, 'html.parser')

In [26]:
new_soup.find("h1",class_ = "lh-copy dark-gray mv1 f4 mh0 b").text.strip()

'Equate 24 Hour Allergy, Cetirizine Hydrochloride Tablets, 10 mg, 45 Count Compare to Zyrtec® Active Ingredient'

In [27]:
new_soup.find("span", class_='inline-flex flex-column').text.strip()

'Now $9.77'

In [44]:
new_soup.find_all("a", class_='pl1 bg-transparent bn lh-solid pa0 sans-serif tc underline inline-button dark-gray pointer f7')[0].text.strip()


'13.6K ratings'

In [48]:
# scrape image
image = new_soup.find("div", class_='relative db').find("img")["src"]

In [49]:
image

'https://i5.walmartimages.com/seo/Equate-24-Hour-Allergy-Cetirizine-Hydrochloride-Tablets-10-mg-45-Count_e153b8b8-d566-4b9d-a82b-9e772e7ef6a9.c079530bf51c47c01c8f61efa260dae2.jpeg?odnHeight=117&odnWidth=117&odnBg=FFFFFF'

---

---

---
------

# WalMart Data

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import random

# -------------------- FUNCTIONS TO EXTRACT DETAILS --------------------

def get_title(soup):
    try:
        title = soup.find("h1",class_ = "lh-copy dark-gray mv1 f4 mh0 b")
        title_string = title.text.strip()
    except AttributeError:
        title_string = ""
    return title_string

def get_price(soup):
    try:
        price = soup.find("span", class_='inline-flex flex-column').text.strip()
    except AttributeError:
        
        price = ""
    return price


def get_link(soup):
    try:
        link = soup.find("a", class_='hide-sibling-opacity')
        if "https://www.walmart.com" in link.get('href'):
                product_link = link.get('href')
        else:
                product_link = "https://www.walmart.com" + link.get('href')
    except AttributeError:
        product_link = ""
    return product_link
    
# function to scrape image of product
def get_image(soup):
    try:
        image = soup.find("div", class_='relative db').find("img")
        image_url = image['src']
    except AttributeError:
        image_url = ""
    return image_url

# -------------------- MAIN SCRIPT START --------------------

if __name__ == '__main__':
    
    # Different User Agents to avoid quick blocking
    USER_AGENTS = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15'
    ]

    BASE_URL = "https://www.walmart.com/search?q=electronics"
    # Dictionary to store data
    d = {"title": [], "price": [], "link": [], "image": []}

    # Loop through pages 1 to 20
    for page in range(1, 26):
        print(f"\nScraping Page {page}...")

        url = BASE_URL + f"&page={page}"

        # Randomize user agent for each page
        headers = {
            'User-Agent': random.choice(USER_AGENTS),
            'Accept-Language': 'en-US, en;q=0.5'
        }

        webpage = requests.get(url, headers=headers)
        soup = BeautifulSoup(webpage.content, "html.parser")

        # Find all product links on the page
        links = soup.find_all("a", attrs={'class': 'hide-sibling-opacity'})

        links_list = []
        for link in links:
            href = link.get('href')
            links_list.append(href)

        print(f"Found {len(links_list)} product links.")

        # Visit each product link
        for link in links_list:
            # product_url = "https://www.walmart.com" + link
            if "https://www.walmart.com" in link:
                product_url = link
            else:
                product_url = "https://www.walmart.com" + link

            try:
                product_headers = {
                    'User-Agent': random.choice(USER_AGENTS),
                    'Accept-Language': 'en-US, en;q=0.5'
                }
                new_webpage = requests.get(product_url, headers=product_headers)
                new_soup = BeautifulSoup(new_webpage.content, "html.parser")

                # Extract and store data
                d['title'].append(get_title(new_soup))
                d['price'].append(get_price(new_soup))
                d['link'].append(get_link(new_soup))
                d['image'].append(get_image(new_soup))

                time.sleep(1)  # Sleep between each product to avoid detection

            except Exception as e:
                print(f"Failed to scrape {product_url}: {e}")
                continue

        time.sleep(2)  # Sleep between pages

    # Save results to CSV
    walmart_df = pd.DataFrame.from_dict(d)
    walmart_df['title'].replace('', np.nan, inplace=True)
    walmart_df = walmart_df.dropna(subset=['title'])  # Remove empty titles
    walmart_df.to_csv("amazon_data.csv", header=True, index=False)

    print("\nScraping complete! Data saved to 'amazon_data.csv'")
    print(walmart_df.head())  # Display the first few rows of the DataFrame


Scraping Page 1...
Found 10 product links.

Scraping Page 2...
Found 11 product links.

Scraping Page 3...
Found 11 product links.

Scraping Page 4...
Found 11 product links.

Scraping Page 5...
Found 11 product links.

Scraping Page 6...
Found 11 product links.

Scraping Page 7...
Found 11 product links.

Scraping Page 8...
Found 11 product links.

Scraping Page 9...
Found 11 product links.

Scraping Page 10...
Found 11 product links.

Scraping Page 11...
Found 11 product links.

Scraping Page 12...
Found 11 product links.

Scraping Page 13...
Found 0 product links.

Scraping Page 14...
Found 0 product links.

Scraping Page 15...
Found 0 product links.

Scraping Page 16...
Found 0 product links.

Scraping Page 17...
Found 0 product links.

Scraping Page 18...
Found 0 product links.

Scraping Page 19...
Found 0 product links.

Scraping Page 20...
Found 0 product links.

Scraping Page 21...
Found 0 product links.

Scraping Page 22...
Found 0 product links.

Scraping Page 23...
Found 0 

C:\Users\Abid\AppData\Local\Temp\ipykernel_13976\471371451.py:151: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  walmart_df['title'].replace('', np.nan, inplace=True)


In [402]:
walmart_df.head(10)

,title,price,link,image
0,Meta Quest 3 Elite Strap with VR Head Strap wi...,$129.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Meta-Quest-3-...
1,Meta Quest 3S 128GB Virtual Reality Headset Ba...,$299.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Meta-Quest-3S...
2,Meta Quest 3 512GB — The Most Powerful Quest —...,$499.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Meta-Quest-3-...
3,"Samsung Galaxy Watch4 40mm Smart Watch, Blueto...",Now $99.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Samsung-Galax...
4,"Straight Talk Moto G Play 4G 2024, 64GB, Blue ...",$29.88,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Straight-Talk...
5,Apple AirPods 4,Now $99.00,,https://i5.walmartimages.com/seo/Apple-AirPods...
6,onn. 32” Class HD (720P) LED Roku Smart Televi...,Now $88.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/onn-32-Class-...
7,"Apple AirTag - 1 Pack, Item Tracker with Apple...",Now $24.95,,https://i5.walmartimages.com/seo/Apple-AirTag-...
8,Acer Chromebook 315 15.6 inch Laptop Intel Pro...,Now $179.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Acer-Chromebo...
9,CyberPowerPC Gamer Master Gaming Desktop AMD R...,Now $809.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/CyberPowerPC-...


In [409]:
# empty to null values of link 
walmart_df['link'].replace('', np.nan, inplace=True)
# drop null values of link
walmart_df = walmart_df.dropna()  # Remove empty titles
walmart_df['price'].replace('', np.nan, inplace=True)
# drop rows with NaN values in the 'price' column with indexing from 0 to onwards
walmart_df.dropna(subset=['price'], inplace=True,)

C:\Users\Abid\AppData\Local\Temp\ipykernel_13976\3388425509.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  walmart_df['link'].replace('', np.nan, inplace=True)
C:\Users\Abid\AppData\Local\Temp\ipykernel_13976\3388425509.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [410]:

walmart_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, 0 to 129
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   115 non-null    object
 1   price   115 non-null    object
 2   link    115 non-null    object
 3   image   115 non-null    object
dtypes: object(4)
memory usage: 4.5+ KB


In [406]:
# remvoe the now and $ sing from price column and convert to float
walmart_df['price'] = walmart_df['price'].str.replace('Now', '')


In [407]:
walmart_df['price'] = walmart_df['price'].str.replace('$', '')

In [408]:
walmart_df['price'] = walmart_df['price'].str.replace(',', '')
walmart_df['price'] = walmart_df['price'].replace('See price in cart', np.nan)

In [411]:
# convert to float
walmart_df['price'] = walmart_df['price'].astype(float)
walmart_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, 0 to 129
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   115 non-null    object 
 1   price   115 non-null    float64
 2   link    115 non-null    object 
 3   image   115 non-null    object 
dtypes: float64(1), object(3)
memory usage: 4.5+ KB


In [412]:
walmart_df.head(20)

,title,price,link,image
0,Meta Quest 3 Elite Strap with VR Head Strap wi...,129.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Meta-Quest-3-...
1,Meta Quest 3S 128GB Virtual Reality Headset Ba...,299.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Meta-Quest-3S...
2,Meta Quest 3 512GB — The Most Powerful Quest —...,499.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Meta-Quest-3-...
3,"Samsung Galaxy Watch4 40mm Smart Watch, Blueto...",99.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Samsung-Galax...
4,"Straight Talk Moto G Play 4G 2024, 64GB, Blue ...",29.88,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Straight-Talk...
6,onn. 32” Class HD (720P) LED Roku Smart Televi...,88.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/onn-32-Class-...
8,Acer Chromebook 315 15.6 inch Laptop Intel Pro...,179.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Acer-Chromebo...
9,CyberPowerPC Gamer Master Gaming Desktop AMD R...,809.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/CyberPowerPC-...
10,Meta Quest 3 Elite Strap with VR Head Strap wi...,129.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Meta-Quest-3-...
11,Meta Quest 3S 128GB Virtual Reality Headset Ba...,299.00,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Meta-Quest-3S...


In [413]:
# save the final dataframe to csv file
walmart_df.to_csv("./walmart data/electronics.csv", header=True, index=False)

In [54]:
import pandas as pd

# Step 1: Load the CSV file
csv_file = 'walmart_data.csv'           # replace with your filename
df = pd.read_csv(csv_file)

# Step 2: Convert to JSON
json_file = 'walmart_data.json'         # desired output filename
df.to_json(json_file, orient='records', indent=4)

print(f"Successfully saved JSON to: {json_file}")

Successfully saved JSON to: walmart_data.json


In [420]:
data1 = pd.read_csv('./walmart data/walmart1.csv')
data2 = pd.read_csv('./walmart data/walmart2.csv')
data3 = pd.read_csv('./walmart data/walmart3.csv')
data4 = pd.read_csv('./walmart data/walmart4.csv')
data5 = pd.read_csv('./walmart data/women_wellness .csv')
data6 = pd.read_csv('./walmart data/usb.csv')
data7 = pd.read_csv('./walmart data/treats.csv')
data8 = pd.read_csv('./walmart data/toys.csv')
data9 = pd.read_csv('./walmart data/tooth_paste.csv')
data10 = pd.read_csv('./walmart data/stationery.csv')
data11 = pd.read_csv('./walmart data/skin_care.csv')
data12 = pd.read_csv('./walmart data/serum.csv')
data13 = pd.read_csv('./walmart data/plant.csv')
data14 = pd.read_csv('./walmart data/pain_management .csv')
data15 = pd.read_csv('./walmart data/ointment.csv')
data16 = pd.read_csv('./walmart data/moisturizer.csv')
data17 = pd.read_csv('./walmart data/makeup.csv')
data18 = pd.read_csv('./walmart data/lipstick.csv')
data19 = pd.read_csv('./walmart data/kitchen.csv')
data20 = pd.read_csv('./walmart data/hair_oil.csv')
data21 = pd.read_csv('./walmart data/hair_color.csv')
data22 = pd.read_csv('./walmart data/groceries.csv')
data23 = pd.read_csv('./walmart data/face_wash.csv')
data24 = pd.read_csv('./walmart data/electronics.csv')
data25 = pd.read_csv('./walmart data/mobiles.csv')
data26 = pd.read_csv('./walmart data/mobiles_accessories.csv')
data27 = pd.read_csv('./walmart data/digestive_health .csv')
data28 = pd.read_csv('./walmart data/diapers_and_wipes.csv')
data29 = pd.read_csv('./walmart data/deodorant.csv')
data30 = pd.read_csv('./walmart data/cream.csv')

In [415]:
data1.head()

,title,price,link,image
0,"Giorgio Armani Acqua Di Gio Eau de Toilette, C...",25.50,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Giorgio-Arman...
1,"Versace Eros Men EDT, Man Eau Fraiche EDT, Pou...",29.75,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Versace-Eros-...
2,"Nautica Voyage Cologne for Men Spray, 3.4 fl oz",12.36,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Nautica-Voyag...
3,"NAUTICA Midnight Voyage Eau De Toilette Spray,...",26.74,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/NAUTICA-Midni...
4,Burberry WEEKEND Eau De Toilette Spray for Men...,28.03,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Burberry-WEEK...


In [422]:
# merge the all dataframes vertically
data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10, data11, data12, data13, data14, data15, data16, data17, data18, data19, data20, data21, data22, data23, data24,data25,data26,data27,data28,data29,data30], ignore_index=True)

In [426]:

data.dropna(inplace=True)
data.isnull().sum()



title    0
price    0
link     0
image    0
dtype: int64

In [427]:
data

,title,price,link,image
0,"Giorgio Armani Acqua Di Gio Eau de Toilette, C...",25.50,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Giorgio-Arman...
1,"Versace Eros Men EDT, Man Eau Fraiche EDT, Pou...",29.75,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Versace-Eros-...
2,"Nautica Voyage Cologne for Men Spray, 3.4 fl oz",12.36,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Nautica-Voyag...
3,"NAUTICA Midnight Voyage Eau De Toilette Spray,...",26.74,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/NAUTICA-Midni...
4,Burberry WEEKEND Eau De Toilette Spray for Men...,28.03,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Burberry-WEEK...
...,...,...,...,...
4341,Amvital Turmeric Cream for Glow - Natural Turm...,9.78,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Amvital-Turme...
4342,Pond's Correcting Facial Cream Clarant B3 Crea...,8.98,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Pond-s-Correc...
4343,Amvital Turmeric Cream for Glow - Natural Turm...,9.78,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Amvital-Turme...
4344,Pond's Correcting Facial Cream Clarant B3 Crea...,8.98,https://www.walmart.com/sp/track?bt=1&eventST=...,https://i5.walmartimages.com/seo/Pond-s-Correc...


In [428]:
data.to_csv('./walmart data/walmart.csv', index=False)

In [430]:
# Step 1: Load the CSV file
csv_file = './walmart data/walmart.csv'           # replace with your filename
df = pd.read_csv(csv_file)

# Step 2: Convert to JSON
json_file = 'walmart_data.json'         # desired output filename
df.to_json(json_file, orient='records', indent=4)

print(f"Successfully saved JSON to: {json_file}")

Successfully saved JSON to: walmart_data.json
